In [4]:
import json

articles = []

with open('data/blog.jsonl') as f:
    for line in f:
        blog = json.loads(line)
        blog['id'] = blog['url']
        articles.append(blog)

len(articles)

795

In [5]:
from ltr.client import ElasticClient
client=ElasticClient()

In [6]:
from ltr.index import rebuild
rebuild(client, index='blog', doc_src=articles)

Reconfig from disk...
Deleted index blog [Status: 200]
Created index blog [Status: 200]
Reindexing...


BulkIndexError: ('100 document(s) failed to index.', [{'index': {'_index': 'blog', '_type': 'movie', '_id': 'https://opensourceconnections.com/blog/2006/06/29/rolling-out-new-blog/', 'status': 400, 'error': {'type': 'illegal_argument_exception', 'reason': 'Rejecting mapping update to [blog] as the final mapping would have more than 1 type: [post, movie]'}, 'data': {'title': 'Rolling out new blog', 'url': 'https://opensourceconnections.com/blog/2006/06/29/rolling-out-new-blog/', 'author': 'eric-pugh', 'content': ' Weve discussed setting up a company-wide blog for a while now, and today I decided to just go ahead and do it. We looked at some of the powerful ones, but realized that simplicity is all we need, and therefore SimpleLog is an obvious answer! \n\n Look for postings about what is going on at OpenSource Connections. \n', 'excerpt': ' Weve discussed setting up a company-wide blog for a while now, and today I decided to just go ahead and do it. We looked at some of the powerful one